In [2]:
import boto3
import dotenv
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [3]:
# .env dosyasını yükleyin
load_dotenv(find_dotenv())

True

In [4]:
aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')

In [5]:
import boto3

# DynamoDB client
dynamodb = boto3.client('dynamodb', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Mevcut tabloları listeleme
response = dynamodb.list_tables()
print(response['TableNames'])


['Video-Games']


In [36]:
import boto3

def scan_table_for_column(table_name, column_name):
    dynamodb = boto3.client('dynamodb', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    
    try:
        # Scan the table
        response = dynamodb.scan(TableName=table_name, Limit=1)
        items = response.get('Items', [])
        
        if items:
            # Check if the column exists in the scanned items
            if column_name in items[0]:
                print(f"Column '{column_name}' exists in the table.")
            else:
                print(f"Column '{column_name}' does not exist in the table.")
        else:
            print(f"No items found in the table '{table_name}'.")
    except Exception as e:
        print(f"Error scanning table: {e}")

if __name__ == "__main__":
    table_name = "Video_Games"
    column_name = "released"
    scan_table_for_column(table_name, column_name)


Column 'released' exists in the table.


In [7]:
import boto3
import concurrent.futures
import pandas as pd

# DynamoDB'ye erişim için Boto3 kullanarak bir client oluşturun
dynamodb = boto3.client('dynamodb', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# PartiQL sorgusu
partiql_statement = 'SELECT slug FROM "Video-Games" WHERE released BETWEEN "2021-01-01" AND "2021-12-31"'

def fetch_page(statement, next_token=None):
    if next_token:
        return dynamodb.execute_statement(Statement=statement, NextToken=next_token)
    else:
        return dynamodb.execute_statement(Statement=statement)

def get_all_items(statement):
    items = []
    response = fetch_page(statement)
    items.extend(response['Items'])

    # Eğer sonuçlar pagination ile geliyorsa, tüm sayfaları dolaşın
    next_tokens = []
    while 'NextToken' in response:
        next_tokens.append(response['NextToken'])
        response = fetch_page(statement, next_token=response['NextToken'])
        items.extend(response['Items'])
        print(f"Fetched {len(response)} items so far...")

    # Using threading to speed up fetching next pages
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_token = {executor.submit(fetch_page, statement, next_token): next_token for next_token in next_tokens}
        for future in concurrent.futures.as_completed(future_to_token):
            response = future.result()
            items.extend(response['Items'])
    return items
    
try:
    # PartiQL sorgusunu çalıştırın ve tüm öğeleri alın
    all_items = get_all_items(partiql_statement)
    
    # Alınan tüm öğeleri pandas DataFrame'e dönüştürme
    df = pd.DataFrame(all_items)
    print(df)

except Exception as e:
    print(f"An error occurred: {e}")


Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 items so far...
Fetched 4 it

KeyboardInterrupt: 

In [ ]:
import boto3
import os
from dotenv import load_dotenv

# .env dosyasını yükleyin
load_dotenv()

# DynamoDB client oluşturma
client = boto3.client(
    'dynamodb',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
    region_name=os.getenv('AWS_DEFAULT_REGION')
)

# Örnek bir satır almak için tarama işlemi
response = client.scan(
    TableName='Video_Games',
    Limit=1  # Sadece bir satır almak için limit belirtiyoruz
)

# Örnek satırdan sütun isimlerini çıkarma
if 'Items' in response and len(response['Items']) > 0:
    sample_item = response['Items'][0]
    column_names = list(sample_item.keys())
    print(f"Sütun İsimleri: {column_names}")
else:
    print("Tabloda veri bulunamadı.")


In [ ]:
['platforms', 'stores', 'playtime', 'name', 'community_rating', 'tba', 'ratings_count', 'ratings',
 'released', 'id', 'tags', 'esrb_rating', 'rating', 'added', 'slug', 
 'added_by_status', 'updated',
 'suggestions_count', 'parent_platforms', 'reviews_count',
 'reviews_text_count', 'genres', 'rating_top']

In [ ]:
import boto3
import os
import pandas as pd
from dotenv import load_dotenv
import decimal

# Load the .env file
load_dotenv()

# Create DynamoDB client
client = boto3.client(
    'dynamodb',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_DEFAULT_REGION')
)

# Scan parameters
table_name = 'Video_Games'
filter_expression = 'begins_with(released, :year)'
expression_attribute_values = {':year': {'S': '2017'}}
columns = [
    'platforms', 'stores', 'playtime', 'game_name', 'community_rating', 'tba', 'ratings_count', 'ratings',
    'released', 'id', 'tags', 'esrb_rating', 'rating', 'added', 'slug',
    'added_by_status', 'updated', 'suggestions_count', 'parent_platforms', 'reviews_count',
    'reviews_text_count', 'genres', 'rating_top'
]

projection_expression = ", ".join(columns)

# Perform the scan operation
response = client.scan(
    TableName=table_name,
    FilterExpression=filter_expression,
    ExpressionAttributeValues=expression_attribute_values,
    ProjectionExpression=projection_expression
)

# Extract the items from the response
items = response['Items']

# Convert data types correctly
for item in items:
    for key, value in item.items():
        if isinstance(value, dict) and 'N' in value:
            item[key] = decimal.Decimal(value['N'])
        elif isinstance(value, dict) and 'BOOL' in value:
            item[key] = value['BOOL']
        # Add other necessary type conversions

# Convert items to a DataFrame
df_games_2017 = pd.DataFrame(items)

# Data cleaning and transformation
df_games_2017['released'] = pd.to_datetime(df_games_2017['released'])  # Convert to date format
# Add other necessary conversions

df_games_2017
